A* for the farmer problem

In [ ]:
import heapq
pq = []
print("Initial pq:", pq)

# push (priority, item)
heapq.heappush(pq, (5, 'task5'))
heapq.heappush(pq, (1, 'task1'))
heapq.heappush(pq, (3, 'task3'))
print("After pushes:", pq)

# pop (lowest priority first)
p = heapq.heappop(pq)
print("Popped:", p)
print("PQ after pop:", pq)

# Note: using (priority, count, item) avoids tie issues when items are not comparable.


In [ ]:
import heapq
import itertools
from typing import Dict, List, Tuple, Optional

# ---------- Node ----------
class Node:
    def __init__(self, state, parent = None, depth: int = 0, cost: float = 0.0):
        self.state  = state
        self.parent = parent
        self.depth  = depth
        self.cost   = cost # cost to this node
        
    def set_parent(self, parent_node, edge_cost: float):
        self.parent = parent_node
        self.depth  = parent_node.depth + 1 if parent_node is not None else 0
        self.cost   = parent_node.cost + edge_cost if parent_node is not None else edge_cost

    def path(self) -> List:
        """Reconstruct state path from start to this node."""
        node = self
        out = []
        while node is not None:
            out.append(node.state)
            node = node.parent
        out.reverse()
        return out

    def __repr__(self): # for debugging, returns a string representation of the Node
        return f"Node(state={self.state!r}, parent={self.parent.state if self.parent else None}, depth={self.depth}, cost={self.cost})"
# ---------- Priority Queue ----------

In [3]:
def get_neighbors(state): # state != goal_state 
    neighbors = []
    new_state = state.copy()
    new_state['Farmer'] = not state['Farmer']

    for item in ['Wolf', 'Goat', 'Cabbage']:
        if state[item] == state['Farmer']:
            new_state[item] = not state[item]
            if is_valid(new_state):
                neighbors.append((new_state, 1)) # step cost is 1
            new_state[item] = state[item]  # revert change for next iteration
    return neighbors

def is_valid(state):
    # check if the state is valid
    # invalid if the goat and wolf are alone without the farmer
    if state['Goat'] == state['Wolf'] and state['Farmer'] != state['Goat']:
        return False
    # invalid if the goat and cabbage are alone without the farmer
    if state['Goat'] == state['Cabbage'] and state['Farmer'] != state['Goat']:
        return False
    return True

In [ ]:
def heuristic_function(state, goal_state): # estimate the cost to the goal from staticmethod
  
  

In [ ]:
# ---------- Uniform Cost Search ----------
def a_star(graph: Dict, start, goal_state, heuristic_func) -> Tuple[Optional[List], float]:
    """
    Returns (path_as_list_of_states, total_cost) or (None, inf) if no path.
    Minimal, correct UCS using Node objects and (priority, counter, node) heap entries.
    """
    counter     = itertools.count() # counter to break ties in priority queue
    start_node  = Node(start, parent = None, depth = 0, cost = 0.0)

    fringe = [] # a priority queue of (cost, count, Node)
    heapq.heappush(fringe, (0.0, next(counter), start_node)) # next(counter) provides unique sequence number
                                                             # breaks the tie
    # cost_so_far = {start: 0.0}    # best known cost to each state
    visited = set()                 # Visited states

    while fringe:
        current_f_value, _, current_node = heapq.heappop(fringe) # remove the node with lowest cost

        # Skip if already finalized
        if current_node.state in visited:
            continue
        # Finalize current state
        visited.add(current_node.state)
        
        # Goal test
        if current_node.state == goal_state:
            return current_node.path(), current_node.cost
        
        # Expand node - add neighbors to fringe
        for neighbor_state, step_cost in get_neighbors(graph, current_node.state):
            new_cost_so_far = current_node.cost + step_cost
        
            # Create Node object the first time we see this state
            if neighbor_state not in visited:
                neighbor_node = Node(neighbor_state)            
                #cost_so_far[neighbor_state] = new_cost
                neighbor_node.set_parent(current_node, edge_cost = step_cost)
                new_f_value = new_cost_so_far + heuristic_function(neighbor_state, goal_state)
                heapq.heappush(fringe, (new_f_value, next(counter), neighbor_node))

    return None, float('inf')  # no path found